# Knapsack Problem - Branch and Bound (Limited Discrepancy Search)

## Problem Definition

Maximize the value of items packed into the knapsack without exceeding its total capacity.

Given $n$ items to choose from, each item $i \in 0...n-1$ has a value $v_{i}$ and a
weight $w_{i}$. The knapsack has a limited capacity $K$. Let $x_{i}$ be a variable that is $1$ if you choose to take item $i$ and $0$ if you leave item $i$ behind.

$$\begin{array}{l l} 
\text{maximise:} \quad 
                    & \sum_{i \in 0...n-1} v_{i}x_{i} \\
\text{subject to:} \quad 
                    & \sum_{i \in 0...n-1} w_{i}x_{i} \leq K \\
                    & x_{i} \in \{0,1\} \quad (i \in 0...n-1)
\end{array}
$$

## Data Format Specification

A knapsack input contains $n + 1$ lines. The first line contains two integers, the first is the number of items in the problem, $n$. The second number is the capacity of the knapsack, $K$. The remaining lines present the data for each of the items. Each line, $i \in 0...n-1$ contains two integers, the item's value $v_i$ followed by its weight $w_i$.

Input Format
<div style="border: 1px solid black">
<samp>n K
v_0 w_0
v_1 w_1
...
v_n-1 w_n-1
</samp>
</div>

The output contains a knapsack solution and is made of two lines. 

The first line contains two values $obj$ and $opt$.

$obj$ is the total value of the items selected to go into the knapsack (i.e. the objective value). $opt$ should be $1$ if your algorithm proved optimality and $0$ otherwise. 

The second line is a list of $n$ $0/1$-values, one for each of the $x_i$ variables. This line encodes the solution.

Output Format
<div style="border: 1px solid black">
<samp>obj opt
x_0 x_1 x_2 ... x_n-1
</samp>
</div>

## Branch and Bound Solution

In [35]:
import heapq
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight', 'perf'])

In [36]:
#Data Preparation
file_location = '.\data\ks_1000_0'

In [37]:
items = []

with open(file_location, 'r') as input_data_file:
    iter_input_data_file = iter(input_data_file.readline, '')
    for line_idx, line in enumerate(iter_input_data_file):
        parts = line.split()
        if line_idx == 0:
            item_count = int(parts[0])
            capacity = int(parts[1])
        else:
            value = int(parts[0])
            weight = int(parts[1])
            perf = float(parts[0]) / float(parts[1])
            items.append(Item(line_idx-1, value, weight, perf))

items.sort(key=lambda item: item.perf, reverse=True)

In [38]:
items[0:10]

[Item(index=642, value=3439, weight=3127, perf=1.0997761432683082),
 Item(index=519, value=13710, weight=12469, perf=1.099526826529794),
 Item(index=336, value=15347, weight=13958, perf=1.0995128241868461),
 Item(index=59, value=6646, weight=6045, perf=1.0994210090984284),
 Item(index=553, value=14781, weight=13446, perf=1.0992860330209728),
 Item(index=525, value=6064, weight=5517, perf=1.0991480877288382),
 Item(index=801, value=2295, weight=2088, perf=1.0991379310344827),
 Item(index=115, value=12721, weight=11574, perf=1.0991014342491792),
 Item(index=504, value=9142, weight=8318, perf=1.0990622745852368),
 Item(index=485, value=11450, weight=10424, perf=1.098426707597851)]

In [39]:
def heuristic(declist, itemlist, capacity, value=0, weight=0):
    '''
    Return upper-bound maximum value for knapsack from decision list, 
    item list (sorted by value/weight) and knapsack capacity.
    '''
    
    if value==0 and weight==0:   #if value and weight aren't input
        value, weight = current_properties(declist, itemlist)
    k = capacity - weight
    k_check = False
    
    if k <= 0:   #check if knapsack at capacity or below
        return value   
    for item in itemlist[len(declist):]:   #iterate over remaining items, calculate maximum possible value
        if (item.weight <= k) and (k > 0):
            k_check = True
            value += item.value
            weight += item.weight
            k -= item.weight
        else:
            if (k_check == True):
                value += (float(item.value) / float(item.weight)) * k
                k = 0
                return value
    
    return value

In [40]:
def current_properties(declist, itemlist):
    '''
    Return current value, weight of knapsack.
    '''
    value = 0
    weight = 0
    for i, dec in enumerate(declist):   #calculate current value, weight of knapsack
        value += dec * itemlist[i].value
        weight += dec * itemlist[i].weight
    return value, weight

In [41]:
Node = namedtuple('Node', ['sortval', 'sortval2', 'heuristic', 'index', 'declist', 'value', 'weight'])

In [42]:
nidx = 0
startnode_heuristic = heuristic([],items,capacity)
startnode = Node(0,-startnode_heuristic, startnode_heuristic ,nidx,[],0,0)

In [43]:
q_nodes, v_decs = [], set()   #q_nodes for nodes yet to be explored, v_decs for set of decision tuples already explored
heapq.heappush(q_nodes, startnode)

In [44]:
b_value = 0
b_weight = 0
b_declist = []
zero_declist = [0] * len(items)

In [45]:
while q_nodes:
    inode = heapq.heappop(q_nodes)
    temp_declist = inode.declist[:]
    #if nidx < 50: print inode
    if nidx % 30000 == 0: print(nidx, b_value, b_weight, len(q_nodes), len(v_decs))
    if nidx % 300000 == 0: print(b_declist)
    #if (tuple(inode.declist) in v_decs) and (nidx != 0):   # if already visited, skip node
    #    continue
    if (inode.value > b_value) and (inode.weight <= capacity):   #if node is better than current best
        b_value = inode.value
        b_weight = inode.weight
        b_declist = inode.declist[:]
        b_declist.extend([0] * (len(items) - len(b_declist)))
    if (inode.value == b_value) and (inode.weight <= capacity) and (len(inode.declist) >= len(b_declist)):
        b_declist = inode.declist[:]
    #v_decs.add(tuple(inode.declist))   #add node to visited list
    #temp_declist.extend([0] * (len(items) - len(temp_declist)))
    #v_decs.add(tuple(temp_declist))
    
    
    if len(inode.declist) == len(items):   #if nil further decisions / children nodes
        continue
    #child nodes
    for dec in [1,0]:
        child_declist = inode.declist[:]
        child_declist.append(dec)
        childtemp_declist = child_declist[:]
        child_value, child_weight = current_properties(child_declist, items)
        child_heuristic = heuristic(child_declist, items, capacity, child_value, child_weight)
        child_sortval = len(child_declist) / max(0.001, float(sum(child_declist)))
        child_node = Node(child_sortval,-child_heuristic, child_heuristic, nidx+1, child_declist, child_value, child_weight)
        if (capacity - child_weight >= 0) and (child_heuristic >= b_value) and (tuple(child_node.declist) not in v_decs):
            heapq.heappush(q_nodes, child_node)
        #else: 
        #    v_decs.add(tuple(child_node.declist))
        #    childtemp_declist.extend([0] * (len(items) - len(childtemp_declist)))
        #    v_decs.add(tuple(childtemp_declist))
        nidx += 1

0 0 0 0 0
[]
30000 109874 99989 1586 0
60000 109883 99994 503 0
90000 109890 99999 53 0
120000 109890 99999 39 0


In [46]:
output_data = str(b_value) + ' ' + str(0) + '\n'

In [47]:
sorted_items = sorted(zip(items, b_declist), key=lambda record: record[0].index, reverse=False)
taken = [b for a, b in sorted_items]

In [48]:
output_data += ' '.join(map(str, taken))

In [49]:
print(output_data)

109899 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

In [50]:
print('Sum Value= ' + str(sum(a.value for (a,b) in sorted_items if b == 1)))

print('Sum Weight= ' + str(sum(a.weight for (a,b) in sorted_items if b == 1)))

print('Capacity= ' + str(capacity))

print('Count Items taken= ' + str(sum(taken)))

Sum Value= 109899
Sum Weight= 99999
Capacity= 100000
Count Items taken= 18
